In [56]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt



postulantesEducacion = pd.read_csv('resources/tp2/tar/fiuba_1_postulantes_educacion.csv')
postulantesGenero = pd.read_csv('resources/tp2/tar/fiuba_2_postulantes_genero_y_edad.csv')
vistas = pd.read_csv('resources/tp2/tar/fiuba_3_vistas.csv')
postulaciones = pd.read_csv('resources/tp2/tar/fiuba_4_postulaciones.csv')
avisosOnline = pd.read_csv('resources/tp2/tar/fiuba_5_avisos_online.csv')
avisosDetalle = pd.read_csv('resources/tp2/tar/fiuba_6_avisos_detalle.csv')

testingSet = pd.read_csv('resources/tp2/test_final_100k.csv')


In [113]:
testingSet.head()

,id,idaviso,idpostulante
0,0,739260,6M9ZQR
1,1,739260,6v1xdL
2,2,739260,ezRKm9
3,3,758580,1Q35ej
4,4,758580,EAN4J6


# Importamos librerias ML

# Transformamos Categorias en Numerical

## PostulantesGenero

In [57]:
#primera estimacion al set de prueba
#primeraPrueba = datosForTesting.loc[:, ['lat', 'lon', 'surface_total_in_m2', 'surface_covered_in_m2']]
#RFRModel = RandomForestRegressor(n_estimators=100, max_features='sqrt')

postulantesGenero['fechanacimiento'] = pd.to_datetime(postulantesGenero['fechanacimiento'], format='%Y-%m-%d', errors='coerce')
postulantesGenero = postulantesGenero.dropna(subset=['fechanacimiento'])

In [ ]:
postulantesGenero.head()

In [ ]:
postulantesGenero[postulantesGenero['idpostulante'] == 'PmqM3oj']  #caso raro

In [58]:
postulantesGeneroOHC = pd.concat([postulantesGenero,pd.get_dummies(postulantesGenero['sexo'], prefix='sexo')],axis=1)

In [59]:
postulantesGeneroOHC = postulantesGeneroOHC[postulantesGeneroOHC['sexo_0.0'] == 0]
postulantesGeneroOHC = postulantesGeneroOHC[['idpostulante','fechanacimiento','sexo_FEM','sexo_MASC','sexo_NO_DECLARA']]

In [ ]:
postulantesGeneroOHC.head()

## Postulantes Educacion

In [60]:
postulantesEducacionOHC = pd.concat([postulantesEducacion,pd.get_dummies(postulantesEducacion['estado'], prefix='estado')],axis=1)
postulantesEducacionOHC = pd.concat([postulantesEducacionOHC,pd.get_dummies(postulantesEducacion['nombre'], prefix='nombre')],axis=1)

In [61]:
postulantesEducacionOHC.drop(['nombre'],axis=1, inplace=True)
postulantesEducacionOHC.drop(['estado'],axis=1, inplace=True)

In [ ]:
postulantesEducacionOHC.head()

## Avisos Detalle

In [ ]:
avisosDetalle.head()

In [62]:
#Para losrimeros casos, me voy a quedar con la zona, tipo de trabajo, nivel laboral y nombre de area
avisosDetalleFiltered = avisosDetalle[['idaviso','nombre_zona','tipo_de_trabajo','nivel_laboral','nombre_area']]

In [63]:
avisosDetalleFiltered = avisosDetalleFiltered.loc[avisosDetalleFiltered['nombre_zona'].notnull(),:]
avisosDetalleFiltered = avisosDetalleFiltered.loc[avisosDetalleFiltered['nombre_area'].notnull(),:]
avisosDetalleFiltered = avisosDetalleFiltered.loc[avisosDetalleFiltered['tipo_de_trabajo'].notnull(),:]
avisosDetalleFiltered = avisosDetalleFiltered.loc[avisosDetalleFiltered['nivel_laboral'].notnull(),:]

In [ ]:
#avisosDetalleFiltered['nombre_area'].unique()
#avisosDetalleFiltered['nombre_zona'].unique()
#avisosDetalleFiltered['nivel_laboral'].unique()
avisosDetalleFiltered['tipo_de_trabajo'].unique()

In [ ]:
avisosDetalleFiltered.head()

In [64]:
avisosDetalleOHC = pd.concat([avisosDetalleFiltered,pd.get_dummies(avisosDetalleFiltered['nombre_zona'], prefix='zona')],axis=1)
avisosDetalleOHC = pd.concat([avisosDetalleOHC,pd.get_dummies(avisosDetalleFiltered['tipo_de_trabajo'], prefix='tipo')],axis=1)
avisosDetalleOHC = pd.concat([avisosDetalleOHC,pd.get_dummies(avisosDetalleFiltered['nivel_laboral'], prefix='nivel')],axis=1)

In [65]:
avisosDetalleOHC.drop(['nombre_zona'],axis=1, inplace=True)
avisosDetalleOHC.drop(['tipo_de_trabajo'],axis=1, inplace=True)
avisosDetalleOHC.drop(['nivel_laboral'],axis=1, inplace=True)

avisosDetalleOHC.drop(['nombre_area'],axis=1, inplace=True)

In [ ]:
avisosDetalleOHC.head()

In [66]:
postulacionesFiltered = postulaciones[['idpostulante','idaviso']]
postulacionesFiltered.count()

idpostulante    4909641
idaviso         4909641
dtype: int64

In [67]:
postulacionesFiltered =  postulacionesFiltered.drop_duplicates()
postulacionesFiltered.count()

idpostulante    4909025
idaviso         4909025
dtype: int64

In [68]:
todoMergeado = pd.merge(postulacionesFiltered,avisosDetalleOHC,how='inner',left_on='idaviso',right_on='idaviso')

In [69]:
todoMergeado = pd.merge(todoMergeado,postulantesEducacionOHC, how='inner', left_on='idpostulante',right_on='idpostulante')

In [70]:
todoMergeado = pd.merge(todoMergeado,postulantesGeneroOHC, how='inner', left_on='idpostulante',right_on='idpostulante')

In [71]:
todoMergeado['sepostulo'] = 1

In [ ]:
todoMergeado.drop_duplicates() 
#CHEQUEAR DUPLICADOS...

In [73]:
#Creo un sample
todoMergeadoSample = todoMergeado.sample(frac=0.3)

In [74]:
##Avisos para los que no se postulo



##avisosNoPostulado = pd.merge(postulaciones,todoMergeado,on=['idpostulante', 'idaviso'] ,how='outer')
avisosNoPostulado = pd.merge(postulaciones,todoMergeadoSample,on=['idpostulante', 'idaviso'] ,how='outer')

In [54]:
avisosNoPostulado.head()

,idaviso,idpostulante,fechapostulacion,zona_Capital Federal,zona_GBA Oeste,zona_Gran Buenos Aires,tipo_Fines de Semana,tipo_Full-time,tipo_Part-time,tipo_Pasantia,...,nombre_Otro,nombre_Posgrado,nombre_Secundario,nombre_Terciario/Técnico,nombre_Universitario,fechanacimiento,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,sepostulo
0,1112248724,NjlD,2018-01-19 07:39:16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
1,1112286523,ZaO5,2018-01-24 15:07:39,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1978-08-10,1.0,0.0,0.0,1.0
2,1112272060,ZaO5,2018-01-24 15:20:10,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1978-08-10,1.0,0.0,0.0,1.0
3,1112288401,ZaO5,2018-01-26 08:37:04,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1978-08-10,1.0,0.0,0.0,1.0
4,1112300563,ZaO5,2018-01-30 13:35:48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN


In [75]:
avisosNoPostulado['sepostulo'] = 0

In [ ]:
avisosNoPostulado.head()

In [ ]:
todoMergeado.head()

In [76]:
#todoMergeadoFINAL = pd.concat(avisosNoPostulado,todoMergeado, ignore_index=True)
todoMergeadoFINAL = pd.concat([avisosNoPostulado, todoMergeadoSample], axis=0, ignore_index=True)

In [ ]:
todoMergeadoFINAL.head()

In [77]:
#lo separo porque sino explota todo
datosForTraining = todoMergeadoFINAL

In [79]:
datosForTraining.head()

,estado_Abandonado,estado_En Curso,estado_Graduado,fechanacimiento,fechapostulacion,idaviso,idpostulante,nivel_Gerencia / Alta Gerencia / Dirección,nivel_Jefe / Supervisor / Responsable,nivel_Junior,...,tipo_Pasantia,tipo_Por Contrato,tipo_Por Horas,tipo_Primer empleo,tipo_Teletrabajo,tipo_Temporario,tipo_Voluntario,zona_Capital Federal,zona_GBA Oeste,zona_Gran Buenos Aires
0,NaN,NaN,NaN,NaT,2018-01-19 07:39:16,1112248724,NjlD,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,0.0,1.0,1978-08-10,2018-01-24 15:07:39,1112286523,ZaO5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,NaN,NaN,NaN,NaT,2018-01-24 15:20:10,1112272060,ZaO5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaT,2018-01-26 08:37:04,1112288401,ZaO5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaT,2018-01-30 13:35:48,1112300563,ZaO5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
datosForTrainingNumericals = datosForTraining.drop(['fechanacimiento','fechapostulacion'],axis=1)

In [85]:
datosForTrainingNumericals.dropna(axis=0,inplace=True, how='any')

In [86]:
datosForTrainingNumericals.set_index('idpostulante','idaviso')

,estado_Abandonado,estado_En Curso,estado_Graduado,idaviso,nivel_Gerencia / Alta Gerencia / Dirección,nivel_Jefe / Supervisor / Responsable,nivel_Junior,nivel_Otro,nivel_Senior / Semi-Senior,nombre_Doctorado,...,tipo_Pasantia,tipo_Por Contrato,tipo_Por Horas,tipo_Primer empleo,tipo_Teletrabajo,tipo_Temporario,tipo_Voluntario,zona_Capital Federal,zona_GBA Oeste,zona_Gran Buenos Aires
idpostulante,,,,,,,,,,,,,,,,,,,,,
ZaO5,0.0,0.0,1.0,1112286523,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
ZaO5,0.0,0.0,1.0,1112317020,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
ZaO5,1.0,0.0,0.0,1112322363,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
ZaO5,0.0,0.0,1.0,1112322363,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
ZaO5,0.0,0.0,1.0,1112327122,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
ZaO5,1.0,0.0,0.0,1112327122,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
ZaO5,1.0,0.0,0.0,1112315062,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
ZaO5,1.0,0.0,0.0,1112346704,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
ZaO5,0.0,0.0,1.0,1112346704,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [87]:
datosForTrainingNumericals = datosForTrainingNumericals.reset_index()

Index(['index', 'estado_Abandonado', 'estado_En Curso', 'estado_Graduado',
       'idaviso', 'idpostulante', 'nivel_Gerencia / Alta Gerencia / Dirección',
       'nivel_Jefe / Supervisor / Responsable', 'nivel_Junior', 'nivel_Otro',
       'nivel_Senior / Semi-Senior', 'nombre_Doctorado', 'nombre_Master',
       'nombre_Otro', 'nombre_Posgrado', 'nombre_Secundario',
       'nombre_Terciario/Técnico', 'nombre_Universitario', 'sepostulo',
       'sexo_FEM', 'sexo_MASC', 'sexo_NO_DECLARA', 'tipo_Fines de Semana',
       'tipo_Full-time', 'tipo_Part-time', 'tipo_Pasantia',
       'tipo_Por Contrato', 'tipo_Por Horas', 'tipo_Primer empleo',
       'tipo_Teletrabajo', 'tipo_Temporario', 'tipo_Voluntario',
       'zona_Capital Federal', 'zona_GBA Oeste', 'zona_Gran Buenos Aires'],
      dtype='object')

In [89]:
datosForTrainingNumericals = datosForTrainingNumericals.drop('index',axis=1)

In [90]:
datosForTrainingNumericals = datosForTrainingNumericals.drop('idaviso',axis=1)

In [91]:
datosForTrainingNumericalsSinPostulacion  = datosForTrainingNumericals.drop('sepostulo',axis=1)

In [96]:
datosForTrainingNumericals = datosForTrainingNumericals.drop('idpostulante',axis=1)

In [98]:
datosForTrainingNumericalsSinPostulacion = datosForTrainingNumericalsSinPostulacion.drop('idpostulante',axis=1)

# Entrenamos Machine Learning

In [103]:
# Load libraries
import pandas
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor

In [100]:
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(datosForTrainingNumericalsSinPostulacion, datosForTrainingNumericals['sepostulo'], test_size=0.2, random_state=7)

In [101]:
models = []
models.append(('LR', LogisticRegression()))
#models.append(('LDA', LinearDiscriminantAnalysis()))
#models.append(('KNN', KNeighborsClassifier()))
#models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
#models.append(('SVM', SVC()))
# evaluate each model in turn
results = []
names = []
for name, model in models:
    kfold = model_selection.KFold(n_splits=10, random_state=7)
    cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

LR: 0.499283 (0.000529)
NB: 0.499447 (0.000968)


In [112]:
from sklearn.metrics import r2_score

models = []
#models.append(('LR', LogisticRegression()))
#models.append(('LDA', LinearDiscriminantAnalysis()))
#models.append(('KNN', KNeighborsClassifier()))
#models.append(('CART', DecisionTreeClassifier()))
models.append(('RF', RandomForestRegressor(n_estimators=100, max_features='sqrt')))
#models.append(('SVM', SVC()))
# evaluate each model in turn
results = []
names = []
for name, model in models:
    model.fit(X_train, Y_train)
    cv_results = r2_score(Y_validation, model.predict(Y_train))
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

ValueError: Expected 2D array, got 1D array instead:
array=[ 0.  0.  0. ...,  1.  1.  0.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

# Prueba 1

In [ ]:
#testingSet.head()

testingSetMerged = pd.merge(testingSet,postulantesEducacionOHC,how='left',on='idpostulante')
testingSetMerged = pd.merge(testingSetMerged,postulantesGeneroOHC,how='left',on='idpostulante')


In [ ]:
testingSetMerged = pd.merge(testingSetMerged,avisosDetalleOHC,how='left',on='idaviso')

In [ ]:
testingSetMerged.columns

In [ ]:
#todoMergeado.columns
todoMergeado.head()

In [ ]:
testingSetMerged['test_set'] =1

In [ ]:
testingSetMerged.drop('id',axis=1,inplace=True)

In [ ]:
todoMergeadoFINAL['train_set'] = 1

In [ ]:
x_train.head()